In [4]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("OPENAI_API_ENDPOINT"), 
  api_key=os.getenv("OPENAI_API_KEY"),  
  api_version=os.getenv("OPENAI_API_VERSION")
)

dalleModel = os.getenv("DALLE_MODEL")
gpt3TurboModel = os.getenv("GPT3_MODEL")
gpt4Model = os.getenv("GPT4_MODEL")
visionModel = os.getenv("VISION_MODEL")
embeddingsModel = os.getenv("EMBEDDINGS_MODEL")

def get_completion(prompt):
    messages = [{"role": "user", "content": prompt}]       
    response = client.chat.completions.create(   
        model=gpt3TurboModel,                                         
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens=1024
    )
    return response.choices[0].message.content


### Exercise 3: Fabrications
Explore what happens when you ask the LLM to return completions for a prompt about a topic that may not exist, or about topics that it may not know about because it was outside it's pre-trained dataset (more recent). See how the response changes if you try a different prompt, or a different model.

In [3]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt)
print(response)

Title: The Martian War of 2076 - A Lesson Plan

Objective: 
To educate students about the Martian War of 2076, its causes, key events, and consequences, fostering critical thinking, historical analysis, and empathy.

Grade Level: 
High School (9th-12th grade)

Duration: 
2-3 class periods (approximately 90 minutes each)

Materials Needed:
1. Access to research materials (books, articles, online resources)
2. Whiteboard or blackboard
3. Markers or chalk
4. Handouts (optional)
5. Multimedia resources (optional)

Lesson Plan:

Day 1:

Introduction (15 minutes)
1. Begin the lesson by engaging students in a brief discussion about science fiction and its influence on society.
2. Introduce the topic of the Martian War of 2076, explaining that it is a fictional event but will be studied as if it were real.
3. Share the lesson objectives and explain the importance of understanding historical events, even if they are fictional.

Causes of the Martian War (30 minutes)
1. Divide students into smal

### Exercise 4: Instruction Based 
Use the "text" variable to set the primary content 
and the "prompt" variable to provide an instruction related to that primary content.

Here we ask the model to summarize the text for a second-grade student

In [4]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt)
print(response)

Jupiter is a really big planet that is fifth from the Sun. It is made of gas and is much smaller than the Sun but bigger than all the other planets combined. People have known about Jupiter for a really long time because it is very bright in the night sky. It is named after a god from ancient Rome. Sometimes, Jupiter is so bright that it can make shadows on Earth. It is usually the third-brightest thing we can see at night, after the Moon and Venus.


### Exercise 5: Complex Prompt 
Try a request that has system, user and assistant messages 
System sets assistant context
User & Assistant messages provide multi-turn conversation context

Note how the assistant personality is set to "sarcastic" in the system context. 
Try using a different personality context. Or try a different series of input/output messages

In [5]:
response = client.chat.completions.create(
    model=gpt3TurboModel,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response.choices[0].message.content)

Oh, it was played in a tiny little bubble called Globe Life Field in Arlington, Texas. Nothing fancy, really.


### Exercise: Explore Your Intuition
The above examples give you patterns that you can use to create new prompts (simple, complex, instruction etc.) - try creating other exercises to explore some of the other ideas we've talked about like examples, cues and more.

In [5]:
def call_openai(num_times, prompt, temperature=0.75, use_seed=False):
    for i in range(num_times):
        if use_seed:
            response = client.chat.completions.create(
                model=gpt3TurboModel,
                messages = [{"role":"system", "content":"You are a helpful assistant."},
                            {"role":"user","content": prompt}],
                    max_tokens=60,
                    seed=123,
                    temperature = temperature
            )
        else:
            response = client.chat.completions.create(
                model=gpt3TurboModel,
                messages = [{"role":"system", "content":"You are a helpful assistant."},
                            {"role":"user","content": prompt}],
                    max_tokens=60,
                    temperature = temperature
            )
        print(response.choices[0].message.content)

In [6]:
call_openai(10, 'The best pet is a ')

subjective topic and may vary depending on individual preferences and lifestyle. Some popular choices for pets include dogs, cats, fish, birds, rabbits, and hamsters. It ultimately depends on what you are looking for in a pet - whether it's companionship, low maintenance, or specific qualities such as
subjective choice that varies from person to person. Some people may prefer dogs as they are loyal and provide companionship, while others may prefer cats for their independence and low-maintenance nature. Other popular pets include fish, birds, rabbits, and guinea pigs. Ultimately, the best pet for you
subjective topic and can vary depending on individual preferences and lifestyles. Some people may prefer dogs as they are loyal, affectionate, and can be trained to perform various tasks. Others may prefer cats for their independence, low maintenance, and ability to provide companionship. Some may prefer smaller pets like ham
subjective preference and ultimately depends on individual circu

In [7]:
call_openai(10, 'The best pet is a ', temperature = 0, use_seed=True)


subjective topic and can vary depending on individual preferences and lifestyles. Some people may prefer dogs as they are loyal, trainable, and provide companionship. Others may prefer cats for their independence, low maintenance, and ability to be left alone for longer periods of time. Other popular pets include birds, fish
subjective topic and can vary depending on individual preferences and lifestyles. Some people may prefer dogs as they are loyal, trainable, and provide companionship. Others may prefer cats for their independence, low maintenance, and ability to be left alone for longer periods of time. Other popular pets include birds, fish
subjective topic and can vary depending on individual preferences and lifestyles. Some people may prefer dogs as they are loyal, trainable, and provide companionship. Others may prefer cats for their independence, low maintenance, and ability to be left alone for longer periods of time. Other popular pets include birds, fish
subjective topic an

In [10]:
response = client.chat.completions.create(
                model=gpt3TurboModel,
                messages = [{"role":"system", "content":"You are a helpful assistant."},
                            {"role":"user","content": "The best pet is a "}],
                    max_tokens=60,
                    n=1
            )

print(response.choices)

[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='subjective topic and largely depends on individual preferences and lifestyles. Some people may prefer dogs as they are loyal and offer companionship, while others may prefer cats for their independent and low-maintenance nature. Other popular pets include birds, fish, rabbits, and hamsters. Ultimately, the best pet for', role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}), Choice(finish_reason='length', index=1, logprobs=None, message=ChatCompletionMessage(content='subjective choice, as it depends on individual preferences and lifestyles. Some people may prefer dogs as pets, as they are known to be loyal and affectionate companions. Others may prefer cats, as t